In [449]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


In [450]:
data = pd.read_csv('data.csv')
data_X = data.iloc[:,0].values
data_y = data.iloc[:,1].values

data.head()

,one,two
0,1.0,1.336431
1,2.0,4.995345
2,3.0,5.043316
3,4.0,7.257684
4,5.0,6.797201


In [451]:
X_train, X_test, y_train, y_test = train_test_split(data_X, data_y, test_size=0.2, random_state=42)

In [452]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='linear', input_shape=(1,)),
    tf.keras.layers.Dense(20, activation='linear'),
    tf.keras.layers.Dense(1)
])

model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=tf.keras.optimizers.Adam())

model.fit(X_train, y_train, epochs=30, batch_size=1)


Epoch 1/30
160/160 [==============================] - 1s 932us/step - loss: 9136.4082
Epoch 2/30
160/160 [==============================] - 0s 900us/step - loss: 389.8906
Epoch 3/30
160/160 [==============================] - 0s 875us/step - loss: 1.2529
Epoch 4/30
160/160 [==============================] - 0s 869us/step - loss: 1.2654
Epoch 5/30
160/160 [==============================] - 0s 899us/step - loss: 1.2977
Epoch 6/30
160/160 [==============================] - 0s 894us/step - loss: 1.2306
Epoch 7/30
160/160 [==============================] - 0s 900us/step - loss: 1.2603
Epoch 8/30
160/160 [==============================] - 0s 872us/step - loss: 1.2167
Epoch 9/30
160/160 [==============================] - 0s 900us/step - loss: 1.3170
Epoch 10/30
160/160 [==============================] - 0s 888us/step - loss: 1.2412
Epoch 11/30
160/160 [==============================] - 0s 900us/step - loss: 1.2642
Epoch 12/30
160/160 [==============================] - 0s 1ms/step - loss: 1.207

In [453]:
mse = model.evaluate(X_test, y_test)
print(f'Test {mse=}')

2/2 [==============================] - 0s 2ms/step - loss: 2.1103
Test mse=2.110267162322998
